In [1]:
import training_module
import modeling_midule
import warnings
warnings.filterwarnings('ignore')

# 각자 환경에 맞게 셋팅

- basic path = 데이터 경로
- path = 모델링 결과를 저장할 경로 // path의 마지막이 "/" 으로 끝나지 않도록 주의!

In [2]:
basic_path = 'D:/kkbox-churn-prediction-challenge/data/test1'

path = 'D:/kkbox-churn-prediction-challenge/test1_result'
members_path = rf'{basic_path}/random_120000_members.parquet'
train_path = rf'{basic_path}/random_120000_train.parquet'
transaction_path = rf'{basic_path}/random_120000_transaction.parquet'
user_logs_path = rf'{basic_path}/random_120000_user_logs.parquet'

In [3]:
train = training_module.TrainingModule()

In [4]:
train.importData(train_path, members_path, transaction_path, user_logs_path)

In [5]:
# 오리지널 / 드랍 (encoding 해야함)
gender_list_train = train.gender()
print(f'gender :{len(gender_list_train)}')

# 그대로 사용 / 15 ~ 64세 와 나머지
original_age_list_train = train.age()


# 오리지널 = 10세 구분 ,  /  오리지널 = 5세 구분 / 오리지널 = 25세 이하 or 이상
# 15 ~ 64 와 나머지 = 10세 구분 / 15 ~ 64 와 나머지 = 5세 구분 / 15 ~ 64 와 나머지 = 25세 이하 or 이상
age_category_list_train = train.ageCategory()

age_list_train = original_age_list_train + age_category_list_train
print(f'age_list_train :{len(age_list_train)}')

# 오리지널 / 이분법
city_list_train = train.cityBinary()
print(f'city :{len(city_list_train)}')

# 연속(합) / 자동구독 여부  
auto_renew_list_train = train.autoRenew()
print(f'autoRenew :{len(auto_renew_list_train)}')

# 지불방식
payment_train = train.paymentMethodId()
print(f'paymentMethodId :{len(payment_train)}')


# 할인받은 기록
discount_train = train.dicountRecord()
print(f'dicountRecord :{len(discount_train)}')

# 년도월 / 년도
init_regist_list_train = train.initRegistration()
print(f'initRegistration :{len(init_regist_list_train)}')


# 최초가입후 멤버쉽 결제기간 : 연thr형 / 카테고리
delay_transaction_list_train = train.delayTran()
print(f'delayTran :{len(delay_transaction_list_train)}')


# 멤버쉽 유지기간
membership_duration_train = train.membershipDuration()
print(f'membershipDuration :{len(membership_duration_train)}')


# 구독 취소여부
is_cancel_train = train.isCancel()
print(f'isCancel :{len(is_cancel_train)}')


# 장기미접속 : 연속형 / 카테고리형
logterm_unconnect_list_train = train.longTermUnconnect()
print(f'longTermUnconnect :{len(logterm_unconnect_list_train)}')


# 유저그룹
user_group_train = train.userLogGroup()
print(f'userLogGroup :{len(user_group_train)}')

gender :2
age_list_train :8
city :2
autoRenew :2
paymentMethodId :119755
dicountRecord :119755
initRegistration :2
delayTran :2
membershipDuration :119755
isCancel :119755
longTermUnconnect :2
userLogGroup :119669


In [6]:
y = train.train

In [7]:
import pandas as pd
from functools import reduce

# 테스트 본격 시작

In [8]:
columns = ['dataframe','model','gender','age','city','autorenew','pay method', 'discount' ,'init regist', 'delay membership', 'membership duration','is cancel','unconnect','user log', 'Train Precision', 'Test Precision', 'Test Recall','Train Recall' , 'Train Matrix','Test Matrix']
result = pd.DataFrame(columns=columns)

count = 0

for gender_num , gender_df in enumerate(gender_list_train):
    for city_num, city_df in enumerate(city_list_train):
        for renew_num, renew_df in enumerate(auto_renew_list_train):
            for unconnect_num, unconnect_df in enumerate(logterm_unconnect_list_train):
                for init_num, init_df in enumerate(init_regist_list_train):
                    for delay_num, delay_df in enumerate(delay_transaction_list_train):
                        for age_num, age_df in enumerate(age_list_train):
                            count += 1
                            df_list = [gender_df, age_df, city_df, renew_df, payment_train, discount_train,
                                       init_df, delay_df,membership_duration_train, is_cancel_train, unconnect_df, 
                                        user_group_train,y]
                            globals()[f"df_{count}"] = reduce(lambda left, right: pd.merge(left, right, on='msno_num'),df_list)
                            

                            ################################### 클래스 생성 ###########################

                            model = modeling_midule.ModelingModule(path=path)

                            ################################### 데이터 프레임 받아오기 ###########################

                            model.getDataFrame(globals()[f"df_{count}"])

                            ################################### 스케일링(전처리)(없으면 빈칸) ###########################

                            ################################### train_test_split ###########################

                            model.trainTestSplit()

                            ################################### 오버 샘플링 (없으면 빈칸) ######################################
                            model.overSampling()

                            ################################### 모델링(알아서) ###########################
                            for i in range(5):
                                if i == 0:
                                    model.modelRandomForest()
                                elif i == 1:
                                    model.modelLightGBM()
                                elif i == 2:
                                    model.modelLogisticRegression()
                                elif i== 3:
                                    model.modelCatboost()
                                elif i == 4:
                                    model.modelGaussianNB()

                                model.saveModel(count)
                                model.modelResult()
                                precision_train, precision_test , recall_train, recall_test, confusion_matrix_train , confusion_matrix_test = model.modelResult()
                                sumary_list = [count, model.model_name, gender_num,age_num,city_num,renew_num,0, 0 ,init_num,
                                                delay_num, 0, 0 ,unconnect_num, 0 , precision_train,
                                                precision_test, recall_train, recall_test,
                                                confusion_matrix_train,confusion_matrix_test]
                                result = result.append(pd.Series(sumary_list , index=result.columns), ignore_index=True)

Split Complete
Learning rate set to 0.09154
0:	learn: 0.6419920	total: 185ms	remaining: 3m 4s
1:	learn: 0.5971396	total: 232ms	remaining: 1m 55s
2:	learn: 0.5651630	total: 290ms	remaining: 1m 36s
3:	learn: 0.5362596	total: 339ms	remaining: 1m 24s
4:	learn: 0.5159153	total: 392ms	remaining: 1m 17s
5:	learn: 0.4981717	total: 452ms	remaining: 1m 14s
6:	learn: 0.4840537	total: 505ms	remaining: 1m 11s
7:	learn: 0.4704712	total: 544ms	remaining: 1m 7s
8:	learn: 0.4541282	total: 584ms	remaining: 1m 4s
9:	learn: 0.4460935	total: 628ms	remaining: 1m 2s
10:	learn: 0.4372744	total: 679ms	remaining: 1m 1s
11:	learn: 0.4289447	total: 739ms	remaining: 1m
12:	learn: 0.4225858	total: 779ms	remaining: 59.1s
13:	learn: 0.4151908	total: 825ms	remaining: 58.1s
14:	learn: 0.4067581	total: 906ms	remaining: 59.5s
15:	learn: 0.3988375	total: 979ms	remaining: 1m
16:	learn: 0.3940139	total: 1.03s	remaining: 59.7s
17:	learn: 0.3892235	total: 1.09s	remaining: 59.8s
18:	learn: 0.3854733	total: 1.15s	remaining: 59.

KeyboardInterrupt: 

In [ ]:
columns = ['dataframe','model','gender','age','city','autorenew','pay method', 'discount' ,'init regist', 'delay membership', 'membership duration','is cancel','unconnect','user log', 'Train Precision', 'Test Precision', 'Train Recall','Test Recall', 'Train F1', 'Test F1' , 'Train Matrix','Test Matrix']
result = pd.DataFrame(columns=columns)

count = 0

for gender_num , gender_df in enumerate(gender_list_train):
    for city_num, city_df in enumerate(city_list_train):
        for renew_num, renew_df in enumerate(auto_renew_list_train):
            for unconnect_num, unconnect_df in enumerate(logterm_unconnect_list_train):
                for init_num, init_df in enumerate(init_regist_list_train):
                    for delay_num, delay_df in enumerate(delay_transaction_list_train):
                        # for age_num, age_df in enumerate(age_list_train):
                        count += 1
                        df_list = [gender_df, age_list_train[0], city_df, renew_df, payment_train, discount_train,
                                       init_df, delay_df,membership_duration_train, is_cancel_train, unconnect_df, 
                                        user_group_train,y]
                        globals()[f"df_{count}"] = reduce(lambda left, right: pd.merge(left, right, on='msno_num'),df_list)
                            

                            ################################### 클래스 생성 ###########################

                        model = modeling_midule.ModelingModule(path=path)

                            ################################### 데이터 프레임 받아오기 ###########################

                        model.getDataFrame(globals()[f"df_{count}"])

                            ################################### 스케일링(전처리)(없으면 빈칸) ###########################

                            ################################### train_test_split ###########################

                        model.trainTestSplit()

                            ################################### 오버 샘플링 (없으면 빈칸) ######################################
                        model.overSampling()

                            ################################### 모델링(알아서) ###########################
                        for i in range(4):
                            if i == 0:
                                model.modelRandomForest()
                            elif i == 1:
                                model.modelLightGBM()
                            elif i== 2:
                                model.modelCatboost()
                            elif i == 3:
                                model.modelXGBoost()

                            model.saveModel(count)
                            model.modelResult()
                            precision_train, precision_test , recall_train, recall_test, f1_train, f1_test, confusion_matrix_train , confusion_matrix_test = model.modelResult()
                            sumary_list = [count, model.model_name, gender_num,0,city_num,renew_num,0, 0 ,init_num,
                                                delay_num, 0, 0 ,unconnect_num, 0 , precision_train,
                                                precision_test, recall_train, recall_test, f1_train, f1_test,
                                                confusion_matrix_train,confusion_matrix_test]
                            result = result.append(pd.Series(sumary_list , index=result.columns), ignore_index=True)

In [17]:
result.sort_values(["Train Recall", "Test Precision"], ascending=False).head()

,dataframe,model,gender,age,city,autorenew,pay method,discount,init regist,delay membership,membership duration,is cancel,unconnect,user log,Train Precision,Test Precision,Test Recall,Train Recall,Train Matrix,Test Matrix
7,2,LogisticRegression,0,1,0,0,0,0,0,0,0,0,0,0,0.706645,0.261286,0.715433,0.703190,"[[58566, 24743], [23707, 59602]]","[[14659, 6169], [921, 2182]]"
2,1,LogisticRegression,0,0,0,0,0,0,0,0,0,0,0,0,0.705442,0.259621,0.716213,0.702224,"[[58395, 24914], [23642, 59667]]","[[14614, 6214], [924, 2179]]"
162,33,LogisticRegression,0,0,0,0,0,0,0,0,0,0,1,0,0.708843,0.262094,0.720943,0.701901,"[[58639, 24670], [23248, 60061]]","[[14696, 6132], [925, 2178]]"
122,25,LogisticRegression,0,0,0,0,0,0,1,1,0,0,0,0,0.706414,0.260007,0.716309,0.701257,"[[58508, 24801], [23634, 59675]]","[[14635, 6193], [927, 2176]]"
82,17,LogisticRegression,0,0,0,0,0,0,1,0,0,0,0,0,0.706450,0.259830,0.716465,0.700612,"[[58507, 24802], [23621, 59688]]","[[14635, 6193], [929, 2174]]"


# 결과 저장

In [10]:
result.to_csv(path+'/result.csv')

In [39]:
result1 = result[result['model'].isin(['LGBM', 'RandomForest', 'Catboost'])]

In [40]:
result1.groupby(['age'])[['Train Recall']].count().sort_values(['Train Recall'])

,Train Recall
age,
6,12
7,12
0,15
1,15
2,15
3,15
4,15
5,15


In [41]:
result1.groupby(['age'])[['Train Recall']].mean().sort_values(['Train Recall'])

,Train Recall
age,
4,0.498636
7,0.498818
6,0.513186
2,0.513675
5,0.514040
3,0.514169
1,0.514900
0,0.517736


In [42]:
result1.groupby(['delay membership'])[['Train Recall']].mean().sort_values(['Train Recall'])

,Train Recall
delay membership,
0,0.499961
1,0.525916


In [43]:
result1.groupby(['gender'])[['Train Recall']].mean().sort_values(['Train Recall'])

,Train Recall
gender,
0,0.510889


In [44]:
result1.groupby(['city'])[['Train Recall']].mean().sort_values(['Train Recall'])

,Train Recall
city,
0,0.510889


In [22]:
result.groupby(['pay method'])[['Train Recall']].sum().sort_values(['Train Recall'])

,Train Recall
pay method,
0,110.567838


In [45]:
result1.groupby(['init regist'])[['Train Recall']].mean().sort_values(['Train Recall'])

,Train Recall
init regist,
1,0.485337
0,0.529473


In [46]:
result1.groupby(['delay membership'])[['Train Recall']].mean().sort_values(['Train Recall'])

,Train Recall
delay membership,
0,0.499961
1,0.525916


In [35]:
result.groupby(['membership duration'])[['Train Recall']].mean().sort_values(['Train Recall'])

,Train Recall
membership duration,
0,0.581936


In [36]:
result.groupby(['is cancel'])[['Train Recall']].mean().sort_values(['Train Recall'])

,Train Recall
is cancel,
0,0.581936


In [37]:
result.groupby(['user log'])[['Train Recall']].mean().sort_values(['Train Recall'])

,Train Recall
user log,
0,0.581936
